In [ ]:
!pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

Extraction des concepts skos avec leurs propriétés

In [ ]:
def pretty_print(df):
    return display(HTML(df.to_html().replace(",","<br>")))
    
def get_preferred_label(html_content):
    preferred_label = html_content.find('h2').text
    return preferred_label

def get_related(html_content):
    p_list = []
    element_list = []
    untreated_related = [x.text for x in html_content.find("a", text="Related").find_parent('th').find_next_sibling('td').find_all('a')]
    for i in range(len(untreated_related)):
        if (i % 2) == 0:
            p_list.append(untreated_related[i])
        else:
            element_list.append(untreated_related[i])
    related = [x[0] + ': ' + x[1] for x in zip(p_list, element_list)]
    return (','.join(related))

def get_narrower(html_content):
    p_list = []
    element_list = []
    untreated_narrower = [x.text for x in html_content.find("a", text="Narrower").find_parent('th').find_next_sibling('td').find_all('a')]
    for i in range(len(untreated_narrower)):
        if (i % 2) == 0:
            p_list.append(untreated_narrower[i])
        else:
            element_list.append(untreated_narrower[i])
    narrower = [x[0] + ': ' + x[1] for x in zip(p_list, element_list)]
    return (','.join(narrower))

In [ ]:
link_list=['http://vocab.nerc.ac.uk/collection/A05/','http://vocab.nerc.ac.uk/collection/P01/','http://vocab.nerc.ac.uk/collection/P07/']
final_dic = {}
for element in link_list:
    print (element)
    links = []
    source = requests.get(element).text
    soup = BeautifulSoup(source, 'html.parser')
    table_rows = soup.find_all('td', class_='onefifty')
    for row in table_rows:
        link = row.find('a')
        if link:
            links.append(link['href'])
    for link in links:
        Identifier_link = requests.get(link).text
        soup = BeautifulSoup(Identifier_link, 'html.parser')
        identifier = link.split('/')[-2]
        print (identifier)
        try:
            preferred_label = get_preferred_label(soup)
        except:
            preferred_label = None
        try:
            related = get_related(soup)
        except:
            related = None
        try:
            narrower = get_narrower(soup)
        except:
            narrower = None
        final_dic[identifier] = {'Preferred Label' : preferred_label, 'Related' : related, 'Narrower' : narrower}
with open("final_output.json", 'w') as fich:
        json.dump(final_dic, fich)

In [ ]:
data =pd.DataFrame.from_dict(final_dic)
pretty_print(data)
data.to_csv(r'scrapped_data_from_NVS.csv',index=False)

In [ ]:
data.to_excel(r'scrapped_data_from_NVS.xlsx', index= False)